# Necessary Modules

In [1]:
# If you are planning to run matplotlib qt, check if you have the following packages; if not, run this cell.
#!pip install scikit-image
#!pip install scikit-learn
#!pip install PyQt6
#!pip install PySide6
#!pip install PyQt5

In [2]:
# Import the following:
from pyvista import imred, tv, spectra, stars, slitmask, image
import numpy as np
import pdb
import copy
import matplotlib.pyplot as plt
import os
from astropy.table import vstack
import pandas as pd
import importlib
from astropy.io import fits

# Directory setup

In [3]:
# put directory name with images here
indir='UT230909'
red=imred.Reducer('KOSMOS',dir=indir)

INSTRUMENT: KOSMOS   config: 
  will use format:  UT230909/*{:04d}.f*.fits*
         gain:  [0.6]    rn: [5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


In [4]:
# Display the contente of the directory
red.log().show_in_notebook(display_length=5)

idx,FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
0,Flat_SEG3G2.0001.fits,2023-09-09T01:31:47.843412,,6:56:00.00,75:00:00.00,2.0
1,Flat_SEG3G2.0002.fits,2023-09-09T01:33:24.544267,,6:56:00.00,75:00:00.00,1.0
2,Flat_SEG3G2.0003.fits,2023-09-09T01:34:47.647398,,6:56:00.00,75:00:00.00,0.5
3,Flat_SEG3R2.0004.fits,2023-09-09T01:38:24.858222,,6:56:00.00,75:00:00.00,0.5
4,Flat_EMPTY_TEST.0005.fits,2023-09-09T01:40:21.135840,,6:56:00.00,75:00:00.00,0.5
5,Flat_SEG3R1.0006.fits,2023-09-09T01:43:41.792202,,6:56:00.00,75:00:00.00,0.5
6,Flat_SEG3R1.0008.fits,2023-09-09T01:47:35.037138,,6:56:00.00,75:00:00.00,0.5
7,Flat_SEG3G2.0009.fits,2023-09-09T01:49:32.885458,,6:56:00.00,75:00:00.00,0.5
8,SEG3G2.0010.fits,2023-09-09T02:19:53.351408,Seg3G2-Mask,21:21:32.11,19:08:59.20,20.0
9,SEG3G2.0011.fits,2023-09-09T02:33:52.513049,Seg3G2-Mask,21:21:34.14,19:09:06.69,20.0


**Setup the TV window**

In [5]:
# Use these lines if you are running the notebook yourself. Matplotlib
# window will open outside the notebook, which is the desired behavior so
# you can have a single display tool, which you should leave open. Other
# plot windows will also appear outside the notebook, which you can close
# as desired
%matplotlib qt
t=tv.TV()
plotinter=True

# following lines only for fully non-interactive demo of notebook
#%matplotlib inline
#plotinter=False
#t=None

# Calibration images.

In [6]:
# Display bias or any calibration image you may want to look at.
bi = red.reduce(74)
if t is not None:
     t.tv(bi)

  Reading file: UT230909\Bias.0074.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


## Master bias.

In [7]:
biastims=[74,75,76,77,78] # Make a list of biases using file numbers.
bias=red.mkbias(biastims,display=None) # Make a master bias.

  Reading file: UT230909\Bias.0074.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0075.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0076.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0077.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0078.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [8]:
# Display master bias.
if t is not None:
    t.tv(bias)

## Master dark

In [9]:
darktims=[94,95,96] # Make a list of darks using file numbers.
dark=red.mkdark(darktims,display=None, clip =5) # Make a master dark.

  Reading file: UT230909\dark.0094.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0095.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0096.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [10]:
# Display dark.
if t is not None:
    t.tv(dark)

## Master flat

In [11]:
flatims=[21,22] # Make a list of flats using file numbers.
flat=red.mkflat(flatims,spec=True) # Make masteer flat.

  Reading file: UT230909\Flat_SEG3G2.0021.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [12]:
# Display flat
if t is not None:
    t.tv(flat)

# Science image reduction

In [13]:
# Reduce a using the calibration images previously defined.
star1=red.reduce(20, crbox='lacosmic', bias=bias, flat=flat, dark=dark, display=t)

  Reading file: UT230909\SEG3G2.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  See bias box (solid outlines applied to dashed regions of the same color), and cross section. 
   To continue, hit space in display window (p for debug) 


Traceback (most recent call last):
  File "C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\tv.py", line 190, in __onEvent
    py, px = np.unravel_index(np.argmin(self.img[ydata-n:ydata+n,xdata-n:xdata+n]),
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py", line 1325, in argmin
    return _wrapfunc(a, 'argmin', axis=axis, out=out, **kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py", line 59, in _wrapfunc
    return bound(*args, **kwds)
           ^^^^^^^^^^^^^^^^^^^^
ValueError: attempt to get argmin of an empty sequence


  subtracting bias...
  subtracting dark...
  zapping CRs with astroscrappy detect_cosmics
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 
  flat fielding...


C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\imred.py:765: RuntimeWarning: divide by zero encountered in divide
  corr.data /= flat.data
C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\imred.py:765: RuntimeWarning: invalid value encountered in divide
  corr.data /= flat.data
C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\imred.py:766: RuntimeWarning: divide by zero encountered in divide
  corr.uncertainty.array /= flat.data


  See flat-fielded image and original with - (minus) key.
   To continue, hit space in display window (p for debug) 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


# Find and display slits on a flat image

In [14]:
# Use a single flat
flat1 = red.reduce(22) # Reading a flat.

  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [15]:
if t is not None:
    t.tv(flat1)

In [16]:
trace=spectra.Trace(transpose=True) # Setup a trace base based on KOMOS (Trancepose = True)
t.tvclear()
bottom,top = trace.findslits(flat1,display=t,thresh=0.5,sn=True) # This does not take mkflat().

In [17]:
vars(trace) # visualize the trace

{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'model': [Polynomial([205.39570053,   6.70214955, -19.58890376], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([353.47561566,   6.31656815, -13.19157469], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([500.31128681,   6.02780702,  -9.3204817 ], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([725.38596237,   6.39430102,  -3.60916434], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([934.65809011,   6.48687832,   1.55820243], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1023.40503861,    6.46322524,    3.67504815], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1089.0796024,    6.4892661,    5.2677399], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1154.51935672,    6.50896277,    6.81061458], d

# Read slit mask from a kms-file

In [18]:
kmsfile= 'kms/Copy of kosmos.23.seg3g2.kms' # File directory.
targets = slitmask.read_kms(kmsfile,sort='YMM') # Reading file.
targets # Visualing file content.

ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str2,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.87,-10.801
TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.61
TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272


## Convert the targets table to a pandas Data Frame

In [19]:
df = targets.to_pandas() # Convertion taking place.
df # A look at your table in panda formate.

,ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
0,TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
1,TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
2,TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
3,TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.870,-10.801
4,TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.610
5,TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
6,TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
7,TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
8,TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272
9,TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.780


## Remove specified rows from Data Frame

In [20]:
# Specify the indices of the rows you want to remove
rows_to_remove = [0,1,2,3,4,5,8,10,12,14,15]
# Remove the specified rows
df_cleaned = df.drop(rows_to_remove)
df_cleaned # Visualize the results

,ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
6,TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
7,TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
9,TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.780
11,TARG104,NN,STRAIGHT,0.9,10.0,0.0,212136.755,190631.54,0.154,1.707,-11.247,25.194
13,TARG102,NN,STRAIGHT,0.9,10.0,0.0,212134.103,190555.71,0.154,1.707,-4.831,31.313


## Filter and process data based on index values

In [21]:
# Give your index value
in_dex = df_cleaned.index.tolist()

# Create a new list to store the filtered lines
filtered_rows = []

for index, line in enumerate(trace.rows):
    if index in in_dex:
        filtered_rows.append(line)

# Replace the original trace.rows with the filtered list
trace.rows = filtered_rows

trace.rows # Visulize the results.

[[1089, 1128], [1154, 1193], [1352, 1391], [1542, 1581], [1681, 1720]]

# Object tracing

In [22]:
# Create your own trace from scratch.
# Don't worry about assigning a value to sc0 because the default is 2028, the center of the KOSMOS lens.
trace1=spectra.Trace(lags=range(-150,150),
                    rows= trace.rows ,transpose=red.transpose, rad=5, degree= 3, sigdegree=3)
vars(trace1) # Visualize your own trace

{'type': 'Polynomial1D',
 'degree': 3,
 'sigdegree': 3,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-150, 150),
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]],
 'model': None,
 'sigmodel': None,
 'sc0': None}

## This cell configures and performs a trace operation on an image using predefined rows and parameters:

In [23]:
#Trace
srow = [] # list to allow for multiple tracing.
for lis in trace.rows:
    # Setting the center for each slit and adding it to the list.
    srow.append(lis[0]+((lis[1]-lis[0])//2))

# rad is setting the width of your trace. It will take the center position to be the star position given taking from DS9.
trace1.trace(star1,srow,skip=50,
                    gaussian = True, display=t, rad= 5) #star1
vars(trace1) # Visulize the trace information

C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\spectra.py:1200: RuntimeWarning: invalid value encountered in divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\spectra.py:1205: RuntimeWarning: invalid value encountered in divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<rad))[0]


  Tracing row: 1700
  See trace. Hit space bar to continue....


{'type': 'Polynomial1D',
 'degree': 3,
 'sigdegree': 3,
 'pix0': 0,
 'spectrum': array([   58.44642639, 32540.70898438,  7469.54345703, ...,
         7217.58984375,  5298.26708984, 10843.49902344]),
 'rad': 5,
 'transpose': True,
 'lags': range(-150, 150),
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]],
 'model': [<Polynomial1D(3, c0=1106.42424301, c1=-0.00066254, c2=0.00000092, c3=0.)>,
  <Polynomial1D(3, c0=1170.48348049, c1=-0.00173742, c2=0.00000101, c3=0.)>,
  <Polynomial1D(3, c0=1374.52056073, c1=-0.00675974, c2=0.00000225, c3=0.)>,
  <Polynomial1D(3, c0=1572.04226392, c1=-0.01195563, c2=0.00000349, c3=0.)>,
  <Polynomial1D(3, c0=1709.69169534, c1=-0.0155156, c2=0.00000431, c3=0.)>],
 'sigmodel': [<Polynomial1D(3, c0=4.00552885, c1=-0.002714, c2=0.00000086, c3=-0.)>,
  <Polynomial1D(3, c0=4.11100655, c1=-0.00247141, c2=0.00000053, c3=0.)>,
  <Polynomial1D(3, c0=4.04102029, c1=-0.0026709, c2=0.00000079, c3=-0.)>,
  <Polynomial1D(3, c0=3.980

## Now extract the sciencee frame

In [24]:
out1 = trace1.extract2d(star1, display=t) # 2D Extraction using the reduced image

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


## Now extract the arc frame

In [25]:
arcs=red.sum([23,24]) #'UT240629/APO32_282857_intNe4s.0001.fits']) # Make a master arc
# Display the lamp
if t is not None:
     t.clear()
     t.tv(arcs)

  Reading file: UT230909\Ne_120s_SEG3G2.0023.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Reading file: UT230909\Ne_360s_SEG3G2.0024.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  combining data with sum....
  calculating uncertainty....


In [26]:
arcec1=trace1.extract2d(arcs, display=t) # 2D Extraction using master arcs.

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


## This cell checks if the number of target objects matches the number of slits and updates FITS headers:

In [27]:
if len(targets) == len(bottom) : # Checking if the conditions is mate.
    for arc,target in zip(arcec1,targets) :
        arc.header['XMM'] = target['XMM']
        arc.header['YMM'] = target['YMM']
else :
    print('ERROR, number of identified slits does not match number of targets')

# This cell performs wavelength calibration for each arc spectrum in 'arcec1' using a predefined wavelength solution file:

In [28]:
for i,arc in enumerate(arcec1) :
    # get an initial guess at the shift from reference using XMM (KOSMOS red center!)
    wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
    nrow=arc.shape[0] # this is referring to the slit width of each slit. 
    print(i)
    shift=int(arc.header['XMM'])
    lags=np.arange(shift-400,shift+400)

    iter = True
    while iter :
        iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,
                            lags=lags,thresh=0.5,
                file='new_wave_lamp/copy_new_neon_red_center.dat',
                rad = 6) # The line computing identification.
        lags=np.arange(-19,19) # 
        plt.close()
        
    # Do the 2D wavelength solution, sampling 5 locations across slits.
    wav.identify(arc,plot=True,nskip=nrow//5,thresh=5)
    plt.close()

  rms:    0.177 Angstroms (50 lines)
0
  cross correlating with reference spectrum using lags between:  -405 394
  Derived pixel shift from input wcal:  [-255.75761163]
  See identified lines.
  rms:    3.609 Angstroms (38 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    3.609 Anstroms
  input from plot window...
  rms:    3.122 Anstroms
  input from plot window...
  rms:    0.078 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -19 18
  Derived pixel shift from input wcal:  [-2.68800946e-08]
  See identified lines.
  rms:    0.072 Angstroms (38 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove li

C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\spectra.py:696: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:    0.086 Angstroms (37 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    0.086 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -300 299
  Derived pixel shift from input wcal for row: 38 0
  See identified lines.
  rms:    0.104
rejecting 2 points from 222 total: 
  rms:    0.091
rejecting 1 points from 222 total: 
  rms:    0.091
rejecting 1 points from 222 total: 
  See 2D wavecal fit. Enter space in plot window to continue

  rms:    0.177 Angstroms (50 lines)
2
  cross correlating with reference spectrum using lags between:  -380 419
  Derived pixel shift from input wcal:  [105.69699078]
  See identified lines.
  rms:    3.258 Angstroms (38 lines)
  Input in plot 

# This cell visualizes and processes spectral data after extraction, involving both plotting and adding wavelength information:

In [29]:
t.clear()
plt.figure()
for i,(o,a) in enumerate(zip(out1,arcec1)) :
    print(o.shape)
    print(a.wave)
    o.add_wave(a.wave) # Adding wavelenght information.
    name = o.header["FILE"].split(".")[0]
    print(name)
    #t.tv(o)
    #t.tv(o.wave)
    plt.plot(o.wave[19],o.data[19], label= f'spec{i}') # 19 center of the slits, but could be any.
    plt.legend()
    #o.write(name + "_20_{:03d}.fits".format(i), overwrite=True) # If you wish to save the 2D spectra with wavelength.

(39, 4096)
[[9348.37132885 9347.34768386 9346.32397608 ... 5264.59099748
  5263.7736761  5262.95651839]
 [9348.42808237 9347.40435253 9346.38056001 ... 5264.56087395
  5263.74344614 5262.92618189]
 [9348.48258963 9347.4587786  9346.43490498 ... 5264.53247104
  5263.71494142 5262.89757523]
 ...
 [9348.99930883 9347.97491712 9346.95046357 ... 5264.59054232
  5263.7722951  5262.95421049]
 [9348.97519682 9347.95085214 9346.92644557 ... 5264.62236118
  5263.80417357 5262.98614863]
 [9348.94883855 9347.92454456 9346.90018862 ... 5264.65590067
  5263.83777727 5263.01981662]]
SEG3G2
(39, 4096)
[[9720.96035573 9719.93208765 9718.903758   ... 5601.19523638
  5600.35827395 5599.52146662]
 [9720.94813877 9719.9198706  9718.8915409  ... 5601.1492121
  5600.31214287 5599.47522865]
 [9720.93645454 9719.90818677 9718.8798575  ... 5601.10597024
  5600.26879927 5599.43178322]
 ...
 [9720.85616306 9719.82820104 9718.80017754 ... 5601.29129226
  5600.45366817 5599.6161986 ]
 [9720.86312427 9719.83517983 9

# 1D extraction

This cell will allow you to extract each star with an appropriate rad. Follow the instructions given by the cell output.

**Notes:**
- The code below allows the user to verify both the extraction and wavelength adjustment processes for validation.
- The rms range that works for me at the moment for the wavelength adjustment process is (1.252 to 0.497). This process is not fully validated, I lack data. Thus, it is at the user's own judgment.
- If the user wishes not to adjust, enter anything as input with the exception of "y," and the code will jump the adjustment process.
- Jon Has just made some updates to the wavelength adjustment process. Therefore, it is recommended to wait till we work through that and integrate it into the workflow. 
- The thresh value matters in the adjustment process; it allows for the number of lines that can be identified in terms of the S/N threshold. 

In [36]:
import matplotlib.pyplot as plt
import copy

importlib.reload(spectra)

def model(x):
    return x * 0.

# Get the beginning value of each slit.
list_1 = [lis[0] for lis in trace.rows]

fig, (ax1, ax2) = plt.subplots(2, 1)

for i, out in enumerate(out1):
    nrow = out.shape[0] // 2  # Get the center of slits.
    center = list_1[i] + nrow  # Center for peak value around the middle of each 2D extracted spectrum.
    
    trace2 = spectra.Trace(transpose=False, sc0=center, degree=3, lags=[-19, 19])
    trace2.rows = [0, out.data.shape[0]]
    trace2.index = [0]

    # Find the brightest peak around the center.
    peak, ind = trace2.findpeak(out, thresh=10, sort=True, width=19, percent=30, method='nearest', axis=1)
    
    if peak:
        def model(x):
            return x * 0. + peak[0]
        trace2.model = [model]

        satisfied = False
        while not satisfied:
            rad = int(input(f"Enter the radius for extraction for slit {i} (current default is 5): "))
            sky_width = rad - 5
            spec, back_spec = trace2.extract(out, rad=rad, back=[[-10 + (-1 * sky_width), -rad], [10 + sky_width, rad]], display=t)
            spec.wave = out.wave[peak]
            back_spec.wave = out.wave[peak]

            print(spec.wave[0].shape, spec.data[0].shape)
            print("Review the extraction results.")
            
            user_response = input("Are you satisfied with the results? Enter 'y' for yes, anything else for no: ").strip().lower()
            if user_response == 'y':
                satisfied = True
        spec.write(f"original_spec_{rad}_{i}_{i:04d}.fits".format(i), overwrite = True) # Saving original spectrum.
        back_spec.write(f"original_back_spec_{rad}_{i}_{i:04d}.fits".format(i), overwrite=True)  # Saving brackground spectrum.
        
        # Optional: Adjust wave calibration
        user1_response = input(f"Do you want to adjust the wavelength solution of star {i}? Enter 'y' for yes and anything else for no: ").strip().lower()
        if user1_response == 'y':
            thresh_val = False
            while not thresh_val:
                thresh = float(input(f"Enter thresh value for wavelength adjustment for star {i}: "))
                swav = copy.deepcopy(wav)  # Create a copy of the original wave solution.
                swav.skyline(back_spec, thresh=thresh, linear=True, inter=plotinter, file='pyvista/python/pyvista/data/sky/skyline.dat')
                
                veri = input("Are you satisfied with the rms? Enter 'y' for yes, anything else for no: ").strip().lower()
                if veri == 'y':
                    thresh_val = True
            
            spec.wave = back_spec.wave
            print(spec.wave)
            print(back_spec.wave)

            spec.write(f"adjust_spec_{rad}_{i}_{i:04d}.fits".format(i), overwrite=True)  # Saving adjusted spec spectrum.
            back_spec.write(f"adjusted_back_spec_{rad}_{i}_{i:04d}.fits".format(i), overwrite=True) # Saving adjuted background spectrum

        
        # Plotting spectra
        ax1.plot(spec.wave[0], spec.data[0], label=f'spec{i}')
        ax1.set_title('Extracted star with sky subtracted')
        ax1.legend(loc="upper right")

        ax2.plot(back_spec.wave[0], back_spec.data[0])
        ax2.set_title('Extracted background sky')

    else:
        print(f'No peak found for slit: {i}')

looking for peaks using 38 pixels around 1108, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]


Enter the radius for extraction for slit 0 (current default is 5):  4


  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
Review the extraction results.


Are you satisfied with the results? Enter 'y' for yes, anything else for no:  y


appending uncertainty
appending bitmask
appending wave
appending uncertainty
appending bitmask
appending wave


...

Enter the radius for extraction for slit 4 (current default is 5):  5


  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
Review the extraction results.


Are you satisfied with the results? Enter 'y' for yes, anything else for no:  y


appending uncertainty
appending bitmask
appending wave
appending uncertainty
appending bitmask
appending wave


Do you want to adjust the wavelength solution of star 4? Enter 'y' for yes and anything else for no:  n


In [ ]:
f2_1 = fits.open('back_spec_20_004.fits')
skyline_file_path = 'pyvista/python/pyvista/data/sky/skyline.dat'
skyline_wavelengths = np.loadtxt(skyline_file_path)

plt.figure()
plt.plot(f2_1[4].data[0], f2_1[1].data[0], )
for wl in skyline_wavelengths:
    plt.axvline(wl, color='red', linestyle='--', alpha=0.5)
#plt.ylabel('Intensity')
plt.title('Sky Spectrum and emission lines Correspondence')
plt.grid(True)
#plt.xlim(7100,8533)
plt.legend()


# Checking the accuracy of wavelength calibration

In [40]:
f2_1 = fits.open('back_spec_20_004.fits')
skyline_file_path = 'pyvista/python/pyvista/data/sky/skyline.dat'
skyline_wavelengths = np.loadtxt(skyline_file_path)

fig, (ax1, ax2) = plt.subplots(2,1)
ax1.plot(f2_1[4].data[0], f2_1[1].data[0],label='range: 6350-7100')
ax2.plot(f2_1[4].data[0], f2_1[1].data[0],label='range: 7100-8533')
for wl in skyline_wavelengths:
    ax1.axvline(wl, color='red', linestyle='--', alpha=0.5)
    ax2.axvline(wl, color='red', linestyle='--', alpha=0.5)
ax1.set_ylabel('Intensity')
ax2.set_ylabel('Intensity')
ax1.set_xlabel('Angstrom')
ax2.set_xlabel('Angstrom')
ax1.set_title('Sky Spectrum and emission lines Correspondence')
ax2.set_title('Sky Spectrum and emission lines Correspondence')
plt.grid(True)
ax1.set_xlim(6350,7100)
ax2.set_xlim(7100,8533)
ax1.legend()
ax2.legend()

# Spectra stacking

In [ ]:
# Define file directories.
from astropy.io import fits
f02 = fits.open('spec_without_sky_17_004.fits')
f03 = fits.open('spec_without_sky_18_004.fits')
f03.info()

In [ ]:
from astropy.nddata import CCDData # Import astropy data class format. 
from ccdproc import Combiner # Import ccdproc combiner for stacking.

In [ ]:
#make a ccddata object out of the flux (pixel count) and uncertainty of each spectra you want to combine
ccd_67 = CCDData(data = f02[1].data[0], uncertainty = f02[2].data[0], unit = 'pixel')
ccd_68 = CCDData(data = f03[1].data[0], uncertainty = f03[2].data[0], unit = 'pixel')
#put them all into Combiner (from ccdproc)
spec_67_69 = Combiner([ccd_67, ccd_68])

#then average combine (it is slightly better than median combining)
avg_test_67_69 = spec_67_69.average_combine()

# Plot and compare.
plt.figure()

plt.plot(f02[4].data[0],f02[1].data[0], label= 'SEG3G2_17')
plt.plot(f03[4].data[0],f03[1].data[0], label= 'SEG3G2_18')
plt.plot(f02[4].data[0], avg_test_67_69.data, label = 'Stacked_spectrum')
plt.legend()
plt.xlabel('Angstrom')
plt.ylabel('Pixel count')

## Save the stacked spectra.

In [ ]:
hdu = fits.PrimaryHDU(avg_test_67_69)
hdul = fits.HDUList([hdu])

# Write to FITS file
hdul.writeto('Stacked_spec.fits', overwrite=True)